# Advanced Numpy

In [2]:
import numpy as np

## ndarray 的结构
**ndarray** =
block of memory + indexing scheme + data type descriptor

* raw data
* how to locate an element
* how to interpret an element

<center>
    ndarray在内存中的结构<br>
    
![图片](img/threefundamental.png)
</center>   

* **memory block**: may be shared, .base, .data
* **data type descriptor**: structured data, sub-arrays, byte order, casting, viewing, .astype(), .view()
* **strided indexing**: strides, C/F-order, slicing w/ integers, as_strided, broadcasting, stride tricks, diag, CPU cache coherence

```C
typedef struct PyArrayObject {
        PyObject_HEAD

        /* Block of memory */
        char *data;

        /* Data type descriptor */
        PyArray_Descr *descr;

        /* Indexing scheme */
        int nd;
        npy_intp *dimensions;
        npy_intp *strides;

        /* Other stuff */
        PyObject *base;
        int flags;
        PyObject *weakreflist;
} PyArrayObject;```


### np.array的若干属性和方法
* shape 数组的形状
* dtype 数组元素的types
* strides 在每个维度中连续两个元素之间的btye数
* data 数据在内存中的表示
* \_\_array_interface\_\_ 包含上面属性的字典
* flags 数据在内存中的信息, 如排列方式, 是否只读等

In [3]:
arr = np.ones((10, 5), dtype = np.int8)

In [5]:
#1. shape
print(arr.shape)

(10, 5)


In [6]:
#2. dtype
print(arr.dtype)

int8


In [29]:
#3. strides
arr.strides
#一般来说 某个维度的stride数目越高, 那么在该维度上计算开销越大

(5, 1)

In [7]:
#4. data
print(bytes(arr.data))

b'\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01'


In [8]:
#全部信息: __array_interface__, flags
print(arr.__array_interface__)

{'data': (140320362750720, False), 'strides': None, 'descr': [('', '|i1')], 'typestr': '|i1', 'shape': (10, 5), 'version': 3}


In [9]:
print(arr.flags)

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False


#### dytpe
dtype 的三个属性
* type
* itemsize <u>返回单个数据的字节数</u>
* byteorder

In [13]:
#type
print(np.dtype(int).type)
print(np.dtype(float).type)
print(np.dtype(np.int8).type)

<class 'numpy.int64'>
<class 'numpy.float64'>
<class 'numpy.int8'>


In [14]:
#itemsize
print(np.dtype(float).itemsize)

8


In [15]:
#byteorder
print(np.dtype(float).byteorder)

=


###### dtype有很多, 但是它们是由几个基类派生来的

In [19]:
ints = np.ones(10, dtype = np.uint16)
floats = np.ones(10, dtype = np.float32)
#用 np.issubdtype函数来判断一个dtype是不是另外一个dtype的派生
print(np.issubdtype(ints.dtype, np.integer))
print(np.issubdtype(floats.dtype, np.floating))

True
True


In [20]:
#用 mro()方法可以查看一个dtype的所有父类
print(np.float64.mro())

[<class 'numpy.float64'>, <class 'numpy.floating'>, <class 'numpy.inexact'>, <class 'numpy.number'>, <class 'numpy.generic'>, <class 'float'>, <class 'object'>]


##### astype 和 view
* <u>astype 返回数组的copy 但是 view不会</u>
* astype recast了数组, 通过改变dtype
* view只是改变数组在内存中的寻址方式

In [25]:
x = np.array([1,2,3,4], dtype = np.float32)
#astype recast了array
y = x.astype(np.int8)
#view不会返回copy
z = x.view(np.int8)
#用base函数判断两个数组是否来自同一块内存
y.base is x, z.base is x

(False, True)

#### order(数组在内存中的存储方式)
* C: last dimensions vary fastest (= smaller strides)
* F: first dimensions vary fastest

In [44]:
#C-order， 按行存储
x = np.array([[1, 2, 3],
              [4, 5, 6]], dtype=np.int16, order='C')
#F-order， 按列存储
y = np.array([[1, 2, 3],
              [4, 5, 6]], dtype=np.int16, order='F')

print(x.strides)
print(y.strides)

# print(bytes(x.data))
# print(bytes(y.data))

(6, 2)
(2, 4)


## array 基本操作

### 索引

In [29]:
x = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]], dtype=np.int8)

In [34]:
#正常索引, 访问第一行第二列的元素
print(x[1, 2])

#你也可以计算byte offset, 移动指针那样索引
print(x.flat[x.strides[0] * 1 + x.strides[1] * 2])

6
6


###### slicing返回数组的view

In [16]:
x = np.array([1,2,3,4,5,6], dtype = np.int32)
y = x[::-1]
print('x:', x)
print('y:', y)
#查看而这是否来自同一块内存
print(y.base is x)
print('stride of x:', x.strides)
print('stride of y:', y.strides)

y = x[2:]
#np.int32占 4个字节, 因此二者内存首地址差了8个字节
print(y.__array_interface__['data'][0] - x.__array_interface__['data'][0])

x: [1 2 3 4 5 6]
y: [6 5 4 3 2 1]
True
stride of x: (4,)
stride of y: (-4,)
8


###### fake dimensions with strides
indexing最终要转换成 byte_offset:<br>
$byte\_offset = stride[0]*index[0] + stride[1]*index[1] + ...$<br>
因此,通过改变strides和shape可以"看上去"改变数组的形状

In [21]:
from numpy.lib.stride_tricks import as_strided
#help(as_strided)
x = np.array([1,2,3,4], dtype = np.int16)
#as_strided 函数给数组分配新的shape和stride
print(as_strided(x, strides = (2*2, ), shape = (2, )))

[1 3]


In [42]:
# a little trick, 注意该函数只返回view
arr = np.array([1, 2, 3, 4], dtype=np.int8)
#第一个维度的stride为0
new_arr = as_strided(arr, shape = (3, 4),strides=(0,1))

print(new_arr)
arr[0] = 10
print(new_arr)

[[1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]]
[[10  2  3  4]
 [10  2  3  4]
 [10  2  3  4]]


In [ ]:
# 

### reshape
* arange
* .T
* flatten 和 ravel

In [6]:
arr = np.arange(8)
print('原始数组:\n',arr)

#reshape
print('改变形状后:\n',arr.reshape((4, 2)))
#第二个参数为 -1, 那么按照第一个参数reshape
print('改变形状后:\n',arr.reshape((2, -1)))

#reshape返回copy
#转置返回view(仅仅交换了stride)
print('reshape((4, 2))后转至:\n',arr.reshape((4, 2)).T)

#flatten, 展开数组, 返回copy
print(arr.reshape((2, -1)).flatten())

#ravel 返回一个view
print(arr.reshape((2, -1)).ravel())

原始数组:
 [0 1 2 3 4 5 6 7]
改变形状后:
 [[0 1]
 [2 3]
 [4 5]
 [6 7]]
改变形状后:
 [[0 1 2 3]
 [4 5 6 7]]
reshape((4, 2))后转至:
 [[0 2 4 6]
 [1 3 5 7]]
[0 1 2 3 4 5 6 7]
[0 1 2 3 4 5 6 7]


#### array的两种存储方式

In [31]:
arr_1 = np.arange(12).reshape((3, 4), order = 'C')
arr_2 = np.arange(12).reshape((3, 4), order = 'F')

In [32]:
arr_1.strides, arr_2.strides

((32, 8), (8, 24))

In [33]:
#C: last dimensions vary fastest (= smaller strides)
#F: first dimensions vary fastest

In [30]:
#row-major
#ravel默认按行展开
arr.ravel()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [31]:
#按列展开
arr.ravel('F')

array([ 0,  4,  8,  1,  5,  9,  2,  6, 10,  3,  7, 11])

### array的拼接和分割

In [33]:
#array的拼接
arr1 = np.array([[1,2,3], [4, 5,6]])
arr2 = np.array([[7,8,9], [10, 11, 12]])

In [34]:
#按行拼接
np.concatenate([arr1, arr2], axis = 0)

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [35]:
#按列拼接
np.concatenate([arr1, arr2], axis = 1)

array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]])

In [37]:
#vstack 按行拼接
np.vstack((arr1, arr2))

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [38]:
#hstack 按列拼接
np.hstack((arr1, arr2))

array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]])

In [41]:
#array的分割
arr = np.random.randn(5, 2)

In [45]:
#arr[0:1,:], arr[1:3,:], arr[3:5,:]
np.split(arr, [1, 3], axis = 0)

[array([[-0.75936684,  0.94663469]]), array([[-0.32441202, -0.54957551],
        [ 0.16795239,  0.48218596]]), array([[-0.59554603,  0.64038069],
        [-1.1933533 , -0.22507649]])]

#### stacking helpers r_和 c_

In [46]:
arr1 = np.arange(6).reshape((3, 2))
arr2 = np.random.randn(3, 2)

In [47]:
np.r_[arr1, arr2]

array([[ 0.        ,  1.        ],
       [ 2.        ,  3.        ],
       [ 4.        ,  5.        ],
       [-0.26846236,  1.0869589 ],
       [-0.54866666,  1.62292246],
       [ 0.90965311,  0.25423043]])

In [48]:
np.c_[arr1, arr2]

array([[ 0.        ,  1.        , -0.26846236,  1.0869589 ],
       [ 2.        ,  3.        , -0.54866666,  1.62292246],
       [ 4.        ,  5.        ,  0.90965311,  0.25423043]])

###### c_和r_转化切片为array

In [51]:
np.r_[1:6, -10:-5]

array([  1,   2,   3,   4,   5, -10,  -9,  -8,  -7,  -6])

In [52]:
np.c_[1:6, -10:-5]

array([[  1, -10],
       [  2,  -9],
       [  3,  -8],
       [  4,  -7],
       [  5,  -6]])

### 重复数组的元素: tile 和 repeat

In [53]:
arr = np.arange(3)

In [55]:
arr.repeat(3)

array([0, 0, 0, 1, 1, 1, 2, 2, 2])

In [56]:
arr.repeat([2, 3, 4])

array([0, 0, 1, 1, 1, 2, 2, 2, 2])

In [60]:
#注意对多维数组你需要传递axis否则它会先展开再repeat
arr = np.random.randn(2, 2)
arr.repeat(2, axis = 0)

array([[ 0.41527045,  1.41827071],
       [ 0.41527045,  1.41827071],
       [-0.82058695, -0.17713913],
       [-0.82058695, -0.17713913]])

In [61]:
arr.repeat(2, axis = 1)

array([[ 0.41527045,  0.41527045,  1.41827071,  1.41827071],
       [-0.82058695, -0.82058695, -0.17713913, -0.17713913]])

In [62]:
arr.repeat([2, 3], axis = 0)

array([[ 0.41527045,  1.41827071],
       [ 0.41527045,  1.41827071],
       [-0.82058695, -0.17713913],
       [-0.82058695, -0.17713913],
       [-0.82058695, -0.17713913]])

In [63]:
arr.repeat(2)

array([ 0.41527045,  0.41527045,  1.41827071,  1.41827071, -0.82058695,
       -0.82058695, -0.17713913, -0.17713913])

In [66]:
#title用来进行块状重复

In [64]:
np.tile(arr, 2)

array([[ 0.41527045,  1.41827071,  0.41527045,  1.41827071],
       [-0.82058695, -0.17713913, -0.82058695, -0.17713913]])

In [65]:
np.tile(arr, (3, 2))

array([[ 0.41527045,  1.41827071,  0.41527045,  1.41827071],
       [-0.82058695, -0.17713913, -0.82058695, -0.17713913],
       [ 0.41527045,  1.41827071,  0.41527045,  1.41827071],
       [-0.82058695, -0.17713913, -0.82058695, -0.17713913],
       [ 0.41527045,  1.41827071,  0.41527045,  1.41827071],
       [-0.82058695, -0.17713913, -0.82058695, -0.17713913]])

### Fancy Indexing: take and put
注意 fancy indexing和切片不同。

In [74]:
arr = np.arange(10) * 100
inds = [7, 1, 2, 6]
arr[inds]

array([700, 100, 200, 600])

In [76]:
#用take来fancy indexing
arr.take(inds)

array([700, 100, 200, 600])

In [79]:
#用 put来fancy assign
#put不接受axis他按照数组在内存中的存储方式来assign
arr.put(inds, 100)
arr

array([  0, 100, 100, 300, 400, 500, 100, 100, 800, 900])

In [81]:
arr.put(inds, [40, 41,42,43])
arr

array([  0,  41,  42, 300, 400, 500,  43,  40, 800, 900])

In [83]:
#take也可以用来在其他维度indexing
arr = np.random.randn(2, 4)
inds = [2, 0, 2, 1]
arr.take(inds, axis = 1)

array([[-0.98892398, -0.67543966, -0.98892398, -1.2678341 ],
       [-1.31775201,  1.22510303, -1.31775201, -0.21922061]])

In [84]:
arr

array([[-0.67543966, -1.2678341 , -0.98892398,  2.20214449],
       [ 1.22510303, -0.21922061, -1.31775201, -0.94948468]])

### 数组广播

#### 广播计算: 两个数组的 trailing dimension matches 或者 有一个维度是1

In [31]:
#一维广播计算
arr = np.arange(5)
print('原始数组:', arr)
print('arr * 5:',arr * 5)

原始数组: [0 1 2 3 4]
arr * 5: [ 0  5 10 15 20]


In [33]:
#二维广播计算
arr = np.random.randn(4, 3)
arr.mean(axis = 0)
print('原始数组:\n',arr)
#广播计算, 按行计算
print('按行广播计算:\n',arr - arr.mean(axis = 0))
#广播计算, 按列计算。注意必须让减数的列维度为1
print('按列广播计算:\n',arr - arr.mean(axis = 1)[:, np.newaxis])

原始数组:
 [[-0.62612816 -1.99816655 -0.55176233]
 [-1.52698824 -0.2903179  -0.55939064]
 [ 1.06678615 -1.46225826  1.31608609]
 [-0.21542459 -0.51429887 -0.48430183]]
按行广播计算:
 [[-0.30068945 -0.93190615 -0.48192015]
 [-1.20154953  0.77594249 -0.48954847]
 [ 1.39222486 -0.39599787  1.38592827]
 [ 0.11001412  0.55196152 -0.41445965]]
按列广播计算:
 [[ 0.43255752 -0.93948087  0.50692335]
 [-0.73475598  0.50191436  0.23284162]
 [ 0.75991483 -1.76912959  1.00921476]
 [ 0.18925051 -0.10962378 -0.07962673]]


###### np.newaxis

In [35]:
arr = np.zeros((4, 4))
print('原始数组:\n', arr)
arr_3d = arr[:, np.newaxis, :]
print('arr[:, np.newaxis, :]\n', arr_3d)
arr_3d = arr[:,:, np.newaxis]
print('arr[:,:, np.newaxis]\n', arr_3d)
arr_3d = arr[np.newaxis,:,:]
print('arr[np.newaxis,:,:]\n', arr_3d)
# arr_1d = np.random.normal(size = 3)

# arr_1d[:, np.newaxis]

# arr_1d[np.newaxis, :]

原始数组:
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
arr[:, np.newaxis, :]
 [[[0. 0. 0. 0.]]

 [[0. 0. 0. 0.]]

 [[0. 0. 0. 0.]]

 [[0. 0. 0. 0.]]]
arr[:,:, np.newaxis]
 [[[0.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [0.]]]
arr[np.newaxis,:,:]
 [[[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]]


###### 广播计算与 as_strided

In [41]:
x = np.array([1,2,3,4], dtype = np.int16)
x2 = as_strided(x, strides=(0, 1*2), shape = (3, 4))
y = np.array([5,6,7], dtype = np.int16)
y2 = as_strided(y, strides=(1*2, 0), shape = (3, 4))
print('x2:\n', x2)
print('y2:\n', y2)
print('实际上是一个外积运算:\n', x2 * y2)
#等价于如下的广播计算:
x[np.newaxis, :] * y[:, np.newaxis]

x2:
 [[1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]]
y2:
 [[5 5 5 5]
 [6 6 6 6]
 [7 7 7 7]]
实际上是一个外积运算:
 [[ 5 10 15 20]
 [ 6 12 18 24]
 [ 7 14 21 28]]


array([[ 5, 10, 15, 20],
       [ 6, 12, 18, 24],
       [ 7, 14, 21, 28]], dtype=int16)

###### 三维的广播计算

In [107]:
arr = np.random.randn(3, 4, 5)

In [109]:
#按照最后一个维度取均值
arr.mean(axis = 2)

array([[-0.39314189, -0.53168205,  0.12120555, -0.5114491 ],
       [-0.4899763 , -0.51777506,  0.11691964,  0.42450305],
       [-0.23595574,  0.1202386 , -0.05738313, -0.11191177]])

In [110]:
#上面的操作等价于
(arr[:, :, 0] + arr[:, :, 1] +\
 arr[:, :, 2] + arr[:, :, 3] +\
 arr[:, :, 4]) / 5

array([[-0.39314189, -0.53168205,  0.12120555, -0.5114491 ],
       [-0.4899763 , -0.51777506,  0.11691964,  0.42450305],
       [-0.23595574,  0.1202386 , -0.05738313, -0.11191177]])

In [111]:
#按照中间的维度取平均值
arr.mean(axis = 1)

array([[ 0.11006853, -0.50117318, -0.24429238, -0.68163187, -0.32680547],
       [ 0.0905641 , -0.58144725,  0.7889928 , -0.81321404, -0.06780644],
       [-0.39950116,  0.15316767, -0.36932333,  0.28486338, -0.02547163]])

In [114]:
#对arr的每一页, 取第零行的长度为5的数组
arr[:, 0, :]

array([[ 0.11676297, -1.44261496, -0.17017801, -1.43704082,  0.96736137],
       [ 0.68599173, -2.035222  , -0.16604593, -1.19799388,  0.26338857],
       [ 0.02739841, -0.45373621, -0.78833031, -0.02116913,  0.05605854]])

In [115]:
#注意最后的维度必须是1
arr - arr.mean(axis = 2)[:, :, np.newaxis]

array([[[ 0.50990486, -1.04947307,  0.22296388, -1.04389893,
          1.36050326],
        [-0.36458632, -0.43387666,  0.42019347, -0.29861833,
          0.67688784],
        [ 0.82521565,  0.77830089, -1.56661309, -0.64535477,
          0.60845133],
        [ 0.78480743,  0.01542363,  1.2613537 ,  0.57641206,
         -2.63799681]],

       [[ 1.17596803, -1.5452457 ,  0.32393038, -0.70801758,
          0.75336487],
        [-0.49574748, -1.50209639,  1.7906608 ,  0.05034505,
          0.15683801],
        [-0.12219346, -0.66922359,  1.33673868, -1.18591766,
          0.64059603],
        [ 0.27055798,  1.85710533,  0.17096999, -0.9429373 ,
         -1.35569601]],

       [[ 0.26335414, -0.21778047, -0.55237457,  0.21478661,
          0.29201428],
        [ 0.2123344 ,  0.58851885, -0.67958476,  0.57719597,
         -0.69846447],
        [-0.86654433,  1.27610114, -1.06500836, -0.19677213,
          0.85222368],
        [-0.92213681, -0.74915681,  1.10468643,  0.82925513,
         -0

###### np.newaxis 返回copy, 下面的方法不损耗性能

In [129]:
indexer = [slice(None)] * arr.ndim

In [130]:
indexer

[slice(None, None, None), slice(None, None, None), slice(None, None, None)]

In [131]:
indexer[1] = np.newaxis

In [132]:
#这样不生成copy
arr - arr.mean(1)[tuple(indexer)]

array([[[ 0.00669444, -0.94144178,  0.07411437, -0.75540895,
          1.29416684],
        [-1.0063369 , -0.46438553,  0.1328038 , -0.14866852,
          0.47201126],
        [ 0.83635267,  1.40067962, -1.20111515,  0.15748264,
          1.05646234],
        [ 0.16328979,  0.0051477 ,  0.99419698,  0.74659483,
         -2.82264044]],

       [[ 0.59542762, -1.45377475, -0.95503872, -0.38477984,
          0.33119501],
        [-1.10408664, -1.43842419,  0.48389295,  0.34578403,
         -0.2931306 ],
        [-0.09583792,  0.0291433 ,  0.66466553, -0.25578398,
          0.82532211],
        [ 0.60449693,  2.86305563, -0.19351976,  0.29477979,
         -0.86338651]],

       [[ 0.42689957, -0.60690387, -0.41900698, -0.30603251,
          0.08153017],
        [ 0.73207416,  0.55558978, -0.19002284,  0.41257119,
         -0.55275424],
        [-0.5244263 ,  1.06555034, -0.75306817, -0.53901865,
          0.82031217],
        [-0.63454742, -1.01423625,  1.36209799,  0.43247997,
         -0

#### 广播赋值

In [133]:
arr = np.zeros((4, 3))

In [134]:
arr[:] = 5

In [135]:
arr

array([[5., 5., 5.],
       [5., 5., 5.],
       [5., 5., 5.],
       [5., 5., 5.]])

In [137]:
col = np.array([1, 2, 3, -4])
arr[:] = col[:, np.newaxis]

In [139]:
arr

array([[ 1.,  1.,  1.],
       [ 2.,  2.,  2.],
       [ 3.,  3.,  3.],
       [-4., -4., -4.]])

In [142]:
arr[:2, :] = np.array([[1.37], [0.5]])

In [143]:
arr[:2, :]

array([[1.37, 1.37, 1.37],
       [0.5 , 0.5 , 0.5 ]])

In [144]:
np.array([[1.37], [0.5]])

array([[1.37],
       [0.5 ]])

### Numpy ufunc

In [145]:
#reduce
arr = np.arange(10).reshape((5, -1))

In [146]:
arr

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]])

In [147]:
np.add.reduce(arr, axis = 0)

array([20, 25])

In [148]:
np.add.reduce(arr, axis = 1)

array([ 1,  5,  9, 13, 17])

In [151]:
arr = np.random.randn(5, 5)
arr[::2, :].sort(1)

In [152]:
arr[:, :-1]< arr[:, 1:]

array([[ True,  True,  True,  True],
       [ True, False, False, False],
       [ True,  True,  True,  True],
       [ True, False,  True,  True],
       [ True,  True,  True,  True]])

In [153]:
np.logical_and.reduce(arr[:, :-1]< arr[:, 1:], axis = 1)

array([ True, False,  True, False,  True])

In [154]:
#accumulate
arr = np.arange(15).reshape((3, -1))

In [156]:
arr

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

In [157]:
#相当于np.cumsum
np.add.accumulate(arr, axis = 1)

array([[ 0,  1,  3,  6, 10],
       [ 5, 11, 18, 26, 35],
       [10, 21, 33, 46, 60]])

In [161]:
#outer,外积运算
arr = np.arange(3).repeat([1, 2, 2])

In [162]:
arr

array([0, 1, 1, 2, 2])

In [163]:
np.multiply.outer(arr, np.arange(5))

array([[0, 0, 0, 0, 0],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 2, 4, 6, 8],
       [0, 2, 4, 6, 8]])

In [168]:
#outer结果的维度是两个输入维度的笛卡尔积
x, y = np.random.randn(3, 4), np.random.randn(5)

In [165]:
result = np.subtract.outer(x, y)

In [167]:
result.shape

(3, 4, 5)

In [169]:
#reduceat "local reduce"或者说groupby reduce

In [171]:
#sum on arr[0:5], arr[5:8], arr[8:]
arr = np.arange(10)
np.add.reduceat(arr, [0, 5, 8])

array([10, 18, 17])

In [172]:
arr = np.multiply.outer(np.arange(4), np.arange(5))

In [173]:
arr

array([[ 0,  0,  0,  0,  0],
       [ 0,  1,  2,  3,  4],
       [ 0,  2,  4,  6,  8],
       [ 0,  3,  6,  9, 12]])

In [174]:
np.add.reduceat(arr, [0, 2, 4], axis = 1)

array([[ 0,  0,  0],
       [ 1,  5,  4],
       [ 2, 10,  8],
       [ 3, 15, 12]])

### 定义自己的 ufunc

In [175]:
#这需要用到 numpy的 C-api

In [177]:
#你也可以用 np.vectorize

def add_elements(x, y):
    return x + y
add_them = np.vectorize(add_elements, otypes = [np.float64])

#### 但是速度会变慢！

In [178]:
%time add_them(np.arange(8), np.arange(8))

CPU times: user 516 µs, sys: 181 µs, total: 697 µs
Wall time: 676 µs


array([ 0.,  2.,  4.,  6.,  8., 10., 12., 14.])

In [179]:
%time np.add(np.arange(8), np.arange(8))

CPU times: user 113 µs, sys: 39 µs, total: 152 µs
Wall time: 109 µs


array([ 0,  2,  4,  6,  8, 10, 12, 14])

### 结构数组

In [180]:
dtypes = [('x', np.float64), ('y', np.int32)]
s_arr = np.array([(1.5, 6), (np.pi, -2)], dtype = dtypes)

In [181]:
s_arr

array([(1.5       ,  6), (3.14159265, -2)],
      dtype=[('x', '<f8'), ('y', '<i4')])

In [182]:
s_arr['x']

array([1.5       , 3.14159265])

In [188]:
#每一个tuple代表一行, 其中每个元素有自己的名字和dtype
s_arr = np.array([(1, 2), (3, 4)],
                 dtype = [('foo', np.int32), ('bar', np.float16)])

In [189]:
s_arr['foo']

array([1, 3], dtype=int32)

In [190]:
#你可以指定每个名字的元素数目，即nested dtypes
dtypes = [('x', np.int64, 3), ('y', np.int32)]
arr = np.zeros(4, dtype = dtypes)

In [192]:
#按元素名字访问
arr['x']

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [194]:
#按行访问
arr[0]

([0, 0, 0], 0)

In [195]:
#你也可以指定 nested array的dtype
dtype = [('x', [('a', 'f8'), ('b', 'f4')]), ('y', np.int32)]
data = np.array([((1, 2), 5), ((3, 4), 6)], dtype = dtype)

In [196]:
data

array([((1., 2.), 5), ((3., 4.), 6)],
      dtype=[('x', [('a', '<f8'), ('b', '<f4')]), ('y', '<i4')])

In [198]:
data['x']

array([(1., 2.), (3., 4.)], dtype=[('a', '<f8'), ('b', '<f4')])

In [199]:
data['y']

array([5, 6], dtype=int32)

### 数组排序

In [200]:
arr = np.random.randn(6)

In [201]:
arr

array([-0.94081852,  0.34049241,  0.28475865, -1.28492955, -0.11694657,
        0.5735146 ])

In [204]:
#inplace sort
arr.sort()

In [205]:
arr

array([-1.28492955, -0.94081852, -0.11694657,  0.28475865,  0.34049241,
        0.5735146 ])

In [214]:
#注意排序是 in-place的
arr = np.random.randn(3, 5)
print(arr)
#arr[:, 0]这是view,因而会改变原数组
arr[:, 0].sort()

[[ 0.14617944 -1.33583336 -0.52761219 -0.3499025  -0.74057631]
 [ 0.81289131  1.47274876  1.43594851 -1.13274329  0.93751369]
 [-0.89498026  0.35595642 -0.58033673  1.1295538  -1.35060811]]


In [215]:
arr

array([[-0.89498026, -1.33583336, -0.52761219, -0.3499025 , -0.74057631],
       [ 0.14617944,  1.47274876,  1.43594851, -1.13274329,  0.93751369],
       [ 0.81289131,  0.35595642, -0.58033673,  1.1295538 , -1.35060811]])

#### 与arr.sort()不同, np.sort返回copy

In [216]:
arr = np.random.randn(5)

In [217]:
arr

array([-0.0930514 , -0.36358344,  0.02492301,  1.20568625,  0.20781697])

In [218]:
np.sort(arr)

array([-0.36358344, -0.0930514 ,  0.02492301,  0.20781697,  1.20568625])

In [219]:
#上面的两种sort函数都可以传递axis
arr = np.random.randn(3, 5)

In [220]:
arr

array([[-0.18670167, -0.0155297 , -0.74630543, -0.52168106, -1.94763642],
       [-0.99297076, -0.46536017, -1.30190459,  0.42981291, -0.49313265],
       [ 0.35163803, -1.09103965,  0.41796421, -0.56048864, -0.94452473]])

In [221]:
#按列排序
np.sort(arr, axis = 1)

array([[-1.94763642, -0.74630543, -0.52168106, -0.18670167, -0.0155297 ],
       [-1.30190459, -0.99297076, -0.49313265, -0.46536017,  0.42981291],
       [-1.09103965, -0.94452473, -0.56048864,  0.35163803,  0.41796421]])

In [223]:
#按行排序
np.sort(arr, axis = 0)

array([[-0.99297076, -1.09103965, -1.30190459, -0.56048864, -1.94763642],
       [-0.18670167, -0.46536017, -0.74630543, -0.52168106, -0.94452473],
       [ 0.35163803, -0.0155297 ,  0.41796421,  0.42981291, -0.49313265]])

#### 注意上面的排序都只产生升序序列, 在实际中 用[::-1]来得到数组的逆序

In [224]:
arr

array([[-0.18670167, -0.0155297 , -0.74630543, -0.52168106, -1.94763642],
       [-0.99297076, -0.46536017, -1.30190459,  0.42981291, -0.49313265],
       [ 0.35163803, -1.09103965,  0.41796421, -0.56048864, -0.94452473]])

In [225]:
arr[:, ::-1]

array([[-1.94763642, -0.52168106, -0.74630543, -0.0155297 , -0.18670167],
       [-0.49313265,  0.42981291, -1.30190459, -0.46536017, -0.99297076],
       [-0.94452473, -0.56048864,  0.41796421, -1.09103965,  0.35163803]])

#### 间接排序 argsort lexsort

In [226]:
values = np.array([5, 0, 1, 3, 2])
indexer = values.argsort()

In [227]:
indexer

array([1, 2, 4, 3, 0])

In [228]:
values.take(indexer)

array([0, 1, 2, 3, 5])

In [233]:
##argsort用于二维数组

In [229]:
arr = np.random.randn(3, 5)
arr[0] = values

In [230]:
arr

array([[ 5.        ,  0.        ,  1.        ,  3.        ,  2.        ],
       [-0.89456983,  0.1434465 , -0.10867673,  0.27256812,  0.86537811],
       [-0.65625343, -0.71521695, -0.2810707 , -0.30994412,  2.05698238]])

In [231]:
arr[0]

array([5., 0., 1., 3., 2.])

In [232]:
arr[:, arr[0].argsort()]

array([[ 0.        ,  1.        ,  2.        ,  3.        ,  5.        ],
       [ 0.1434465 , -0.10867673,  0.86537811,  0.27256812, -0.89456983],
       [-0.71521695, -0.2810707 ,  2.05698238, -0.30994412, -0.65625343]])

In [238]:
#下面演示lexsort
#lexsort执行字典排序,注意last_name是第一序!!!!

In [235]:
first_name = np.array(['Bob', 'Jane', 'Steve', 'Bill', 'Barbara'])
last_name = np.array(['Jones', 'Arnold', 'Arnold', 'Jones', 'Walters'])

In [236]:
sorter = np.lexsort((first_name, last_name))

In [237]:
sorter

array([1, 2, 3, 0, 4])

In [239]:
last_name[sorter]

array(['Arnold', 'Arnold', 'Jones', 'Jones', 'Walters'], dtype='<U7')

In [240]:
first_name[sorter]

array(['Jane', 'Steve', 'Bill', 'Bob', 'Barbara'], dtype='<U7')

#### 其他排序算法

In [241]:
values = np.array(['2:first', '2:second', '1:first',
                   '1:second', '1:third'])
keys = np.array([2, 2, 1, 1, 1])

In [245]:
#唯一的稳定的排序算法: 归并排序

In [242]:
indexer = keys.argsort(kind = 'mergesort')

In [243]:
indexer

array([2, 3, 4, 0, 1])

In [244]:
values.take(indexer)

array(['1:first', '1:second', '1:third', '2:first', '2:second'],
      dtype='<U8')

#### 部分排序算法

In [246]:
arr = np.random.randn(20)

In [247]:
arr

array([ 0.92652057,  0.44722123,  0.2036894 ,  0.58230657, -0.44776536,
       -0.8188305 , -0.62248695,  0.28561771,  1.9441149 , -1.10004785,
       -0.56424707,  0.21446351,  1.82571748,  1.29800435, -0.23814031,
        0.16192258,  1.44152247, -0.01066425,  0.8598674 , -0.37920316])

In [249]:
#返回的数组中前三个元素是top3 smallest
np.partition(arr, 3)

array([-0.62248695, -0.8188305 , -1.10004785, -0.56424707, -0.44776536,
       -0.37920316,  0.58230657,  0.28561771,  1.9441149 ,  0.44722123,
        0.2036894 ,  0.21446351,  1.82571748,  1.29800435, -0.23814031,
        0.16192258,  1.44152247, -0.01066425,  0.8598674 ,  0.92652057])

In [254]:
#返回index, 其中前三个索引是top3 smallest的索引
index = np.argpartition(arr, 3)

In [255]:
index

array([ 6,  5,  9, 10,  4, 19,  3,  7,  8,  1,  2, 11, 12, 13, 14, 15, 16,
       17, 18,  0])

In [256]:
np.take(arr, index)

array([-0.62248695, -0.8188305 , -1.10004785, -0.56424707, -0.44776536,
       -0.37920316,  0.58230657,  0.28561771,  1.9441149 ,  0.44722123,
        0.2036894 ,  0.21446351,  1.82571748,  1.29800435, -0.23814031,
        0.16192258,  1.44152247, -0.01066425,  0.8598674 ,  0.92652057])

In [257]:
# np.searchsorted

In [258]:
arr = np.array([0, 1, 7, 12, 15])

In [260]:
#顾名思义, 此方法在一个已经排序的数组上
#执行二分搜索, 返回被搜索元素的下标
arr.searchsorted(7)

2

In [261]:
arr = np.array([0,0,0,1,1,1,1])

In [263]:
#返回左起第一个匹配的元素的下标
arr.searchsorted([0, 1])

array([0, 3])

In [264]:
#你也可以改变匹配策略
arr.searchsorted([0, 1], side = 'right')

array([3, 7])

In [273]:
#一个简单的例子
#这个用法要牢记！！！

In [268]:
data = np.floor(np.random.uniform(0, 10000, size = 50))

In [269]:
data

array([2877., 7349., 4782., 9269.,  388., 6024.,  977., 7085., 2368.,
       2405., 1869., 6930., 6549., 1394., 5041., 2459., 8893., 1952.,
       6484., 4525., 7604., 9988.,   74., 4010., 5005., 1903., 9651.,
       5161., 9744., 6802., 3043., 7187., 3258.,  693., 3293.,  284.,
       4751., 3673., 7120., 4640., 6006., 7446., 3249., 9279., 6793.,
        798., 3291., 4643., 2476., 5324.])

In [270]:
bins = np.array([0, 100, 1000, 5000, 10000])

In [276]:
#返回值 3意味着, 2877 在 1000到5000之间
labels = bins.searchsorted(data)

In [277]:
labels

array([3, 4, 3, 4, 2, 4, 2, 4, 3, 3, 3, 4, 4, 3, 4, 3, 4, 3, 4, 3, 4, 4,
       1, 3, 4, 3, 4, 4, 4, 4, 3, 4, 3, 2, 3, 2, 3, 3, 4, 3, 4, 4, 3, 4,
       4, 2, 3, 3, 3, 4])

In [278]:
import pandas as pd
pd.Series(data).groupby(labels).mean()

1      74.000000
2     628.000000
3    3183.857143
4    7249.304348
dtype: float64

### 数组输入输出

In [279]:
#memory-mapped files

In [282]:
#create a new memory map
mmap = np.memmap('mymap', dtype = np.float64,
                 mode = 'w+', shape = (10000, 10000))

In [283]:
mmap

memmap([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [284]:
#切片 memmap返回一个view on disk
section = mmap[:5, :]

In [285]:
section

memmap([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [286]:
#对这个view写入会buffered到memory里
section[:] = np.random.randn(5, 10000)

In [287]:
#通过flush操作, 写入磁盘中

In [290]:
mmap.flush()

In [292]:
del mmap